## State of Washington - Licensee - Darling Growers

* UBI: 603513879

We'll be using the [`cannapy`](https://github.com/CannabisData/cannapy) library to access the portal data.  `cannapy` aims to provide an abstract interface for accessing and working with *Cannabis* data from around the world.  It utilizes [xmunoz](https://github.com/xmunoz)'s [`sodapy`](https://github.com/xmunoz/sodapy) client to access Socrata-based open data portals and can return data loaded into [Pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

### Dataset: Licensed Businesses

* Canonical Dataset ID: **bhbp-x4eb**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/Licensing/Licensed-Businesses/u3zh-ri66
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/bhbp-x4eb

In [1]:
import time
import cannapy.us.wa.wslcb.portal as wslcb
import pandas as pd

In [2]:
# Specify your own Socrata App Token if you plan to experiment
app_token = 'XaB9MBqc81C3KT4Vps6Wh5LZt'

# Instantiate a cannapy interface to the WSLCB open data portal
portal = wslcb.WSLCBPortal(app_token)

# We'll be using the Licensed Businesses dataset
dataset_id = 'bhbp-x4eb'

# And we're looking for data on a particular licensee
licensee_ubi = '603513879'

In [3]:
# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Thu Dec 14 16:37:18 2017


In [4]:
# Retrieve the dataset preloaded into a Pandas DataFrame
licenses = portal.get_dataframe(dataset_id)

In [5]:
# The UBI column uniquely identifies each licensee, but obscures ownership of multiple licenses by the same entity.
# Let's break that column apart into its constituent parts:
# Unified Business Identifier (UBI): first nine digits
# Business ID Number: next three digits
# Location Number: last four digits
df_v2 = licenses.rename(columns={'ubi': 'ubi_source'})
df_v2['ubi'] = df_v2.ubi_source.str[0:9]
df_v2['ubi_business_id'] = df_v2.ubi_source.str[9:12]
df_v2['ubi_location'] = df_v2.ubi_source.str[12:]
licensee_licenses = df_v2.loc[df_v2['ubi'] == licensee_ubi]
licensee_licenses

,license,type,createdate,active,organization,address,address_line_2,city,state,zip,county,dayphone,ubi_source,ubi,ubi_business_id,ubi_location
753,424381,MARIJUANA PRODUCER TIER 2/MARIJUANA PROCESSOR,20171103,ACTIVE (ISSUED),DARLING GROWERS,3900 BAY RD,NaN,FERNDALE,WA,982489694,WHATCOM,5039018525,6035138790010004,603513879,001,0004


### Dataset: Enforcement Visits

* Canonical Dataset ID: **w7wg-8m52**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/dataset/Enforcement-Visits-Dataset/jizx-thwg
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/w7wg-8m52

In [6]:
# Let's see how many enforcement visits the licensee has hosted
dataset_id = 'w7wg-8m52'

# Select the licensee's license number
# TODO: find a way to do this without hardcoding the row number
licensee_license_number = licensee_licenses.loc[753, 'license']

# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Thu Nov 16 10:55:40 2017


In [7]:
# Retrieve the dataset preloaded into a Pandas DataFrame
enforcement_visits = portal.get_dataframe(dataset_id)

# Suppress the chained assignment warning: https://stackoverflow.com/a/20627316/7622699
pd.options.mode.chained_assignment = None

# Pull aside the enforcement visits by the selected licensee
licensee_enforcement_visits = enforcement_visits.loc[enforcement_visits['license_number'] == licensee_license_number]

# Sort the DataFrame by 'date'
licensee_enforcement_visits.sort_values(by='date', inplace=True)

licensee_enforcement_visits

,date,license_number,city_name,county_name,activity
8449,2017-03-10T00:00:00.000,424381,UNINCORP. AREAS,WHATCOM,Marijuana Premises Check
29,2017-06-05T00:00:00.000,424381,UNINCORP. AREAS,WHATCOM,Marijuana Premises Check
1748,2017-08-25T00:00:00.000,424381,UNINCORP. AREAS,WHATCOM,Marijuana Premises Check
2756,2017-10-31T00:00:00.000,424381,UNINCORP. AREAS,WHATCOM,Marijuana Premises Check


### Dataset: Violations

* Canonical Dataset ID: **dgm4-3cm6**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/dataset/Violations-Dataset/dx3i-tzh2
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/dgm4-3cm6

In [8]:
# Let's pull up all of the licensee's violations
dataset_id = 'dgm4-3cm6'

# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Thu Nov 16 10:54:40 2017


In [9]:
# Retrieve the dataset preloaded into a Pandas DataFrame
violations = portal.get_dataframe(dataset_id)

# Pull aside the violations by the selected licensee
licensee_violations = violations.loc[violations['license_number'] == licensee_license_number]

# Sort the DataFrame by 'visit_date'
licensee_violations.sort_values(by='visit_date', inplace=True)

licensee_violations

,visit_date,license_number,county_name,city_name,case,violation_code,wac_code,penalty_type
